<a id="title_ID"></a>
# Search The TESS Input Catalog Centered On HD 209458.

This notebook tutorial demonstrates how to do a cone search centered on the location of a target (in this example, the star HD 209458) in the TESS Input Catalog.  We will then select a subset of the returned targets based on their Luminosity Class values.  Finally, we will identify the TIC object that is closest on the sky to HD 209458 as a means to determine the star's TIC number.  This tutorial uses Python and the astroquery module.  Information on how to install astroquery for MAST can be found on the [Astroquery site](http://astroquery.readthedocs.io/).

In [1]:
from astroquery.mast import Catalogs
import numpy as np

## Setup The Search Parameters

Let's start by defining the name of our target and our desired search radius in degrees.

In [2]:
target_name = "HD 209458"
search_radius_deg = 0.2

# Astroquery Catalog Cone Search

We will now use the Catalogs `query_object` function to do a cone search centered on HD 209458 and return the rows from the TIC catalog.  The function makes use of MAST's name resolver to convert a target name (as a string) into coordinates (Right Ascension and Declination).  Many types of ID's are possible, including any string that can be resolved by Simbad or NED, as well as any target in the Kepler Input Catalog, K2 EPIC Catalog, or the TESS Input Catalog.  The function also can accept a search radius size (specified in degrees), and it needs to know which catalog you want to query.  Catalog options are listed on [this page](https://astroquery.readthedocs.io/en/latest/mast/mast.html#catalog-queries).

In the second line,  we print out the numbers of returned rows.

In [3]:
# Query the TESS Input Catalog centered on HD 209458 with a 0.2 degree radius.
catalogTIC = Catalogs.query_object(target_name, radius=search_radius_deg, catalog="TIC")

# Print out the number of returned rows.
print("Number of TIC objects within %f deg of %s: %u" % (search_radius_deg, target_name, len(catalogTIC)))

Number of TIC objects within 0.200000 deg of HD 209458: 1345


## Examine The Returned Table

Let's take a look at the table returned from the query.  We see the returned object is an astropy Table.  We can also examine the columns available.

In [4]:
# What type of objects is the returned result?
print(type(catalogTIC))

<class 'astropy.table.table.Table'>


In [5]:
# What columns are available from the TIC?
print(catalogTIC.columns)

<TableColumns names=('ID','ra','dec','pmRA','pmDEC','Tmag','objType','typeSrc','version','HIP','TYC','UCAC','TWOMASS','SDSS','ALLWISE','GAIA','APASS','KIC','POSflag','e_pmRA','e_pmDEC','PMflag','plx','e_plx','PARflag','gallong','gallat','eclong','eclat','Bmag','e_Bmag','Vmag','e_Vmag','umag','e_umag','gmag','e_gmag','rmag','e_rmag','imag','e_imag','zmag','e_zmag','Jmag','e_Jmag','Hmag','e_Hmag','Kmag','e_Kmag','TWOMflag','prox','w1mag','e_w1mag','w2mag','e_w2mag','w3mag','e_w3mag','w4mag','e_w4mag','GAIAmag','e_GAIAmag','e_Tmag','TESSflag','SPFlag','Teff','e_Teff','logg','e_logg','MH','e_MH','rad','e_rad','mass','e_mass','rho','e_rho','lumclass','lum','e_lum','d','e_d','ebv','e_ebv','numcont','contratio','disposition','duplicate_id','priority','eneg_EBV','epos_EBV','EBVflag','eneg_Mass','epos_Mass','eneg_Rad','epos_Rad','eneg_rho','epos_rho','eneg_logg','epos_logg','eneg_lum','epos_lum','eneg_dist','epos_dist','distflag','eneg_Teff','epos_Teff','TeffFlag','gaiabp','e_gaiabp','gaiarp','

## Select Dwarf Stars Based On Luminosity Class

Let's select a subset of the returned objects based on the "lumclass" column.  This column classifies sources in the TIC as "giant" or "dwarf".  For more information on how the columns of the TIC are populated, consult the [TESS Input Catalog paper](http://adsabs.harvard.edu/abs/2017arXiv170600495S) and the [TIC Live Release Notes](https://outerspace.stsci.edu/display/TESS/TIC+and+CTL+Data+Release+Notes+Home+Page).

In [6]:
# Identify where the luminosity class is set to "DWARF".
where_dwarfs = np.where(catalogTIC['lumclass'] == 'DWARF')[0]

# Let's also find out where the luminosity class is set to "GIANT".
where_giants = np.where(catalogTIC['lumclass'] == 'GIANT')[0]

# Let's print out the number of rows of each type within the returned objects from our search.
print("Number of objects classified as 'DWARF' within %f deg of %s: %u" %
      (search_radius_deg, target_name, len(where_dwarfs)))
print("Number of objects classified as 'GIANT' within %f deg of %s: %u" %
      (search_radius_deg, target_name, len(where_giants)))

Number of objects classified as 'DWARF' within 0.200000 deg of HD 209458: 903
Number of objects classified as 'GIANT' within 0.200000 deg of HD 209458: 28


## Find The Closest TIC ID To HD 209458

Let's now find the star in the TESS Input Catalog that is closest to our target. The table produced by the astroquery request is sorted by the column "dstArcSec" by default.  This column gives the distance in arcseconds from the requested target's position returned by the name resolver.  It should be the first row if you haven't sorted the table yet, but let's not assume that and instead find the row with the smallest "dstArcSec".

In [7]:
where_closest = np.argmin(catalogTIC['dstArcSec'])

print("Closest TIC ID to %s: TIC %s, separation of %f arcsec. and a TESS mag. of %f"%
      (target_name, catalogTIC['ID'][where_closest], catalogTIC['dstArcSec'][where_closest],
      catalogTIC['Tmag'][where_closest]))

Closest TIC ID to HD 209458: TIC 420814525, separation of 0.003237 arcsec. and a TESS mag. of 7.127400


Looks like a good match, we know that HD 209458 is a fairly bright star, and the TIC coordinate is very close to the coordinate from the name resolver.

## About this Notebook

**Author:** Scott W. Fleming, STScI Archive Scientist

**Updated On:** 2018-11-27

[Top of Page](#title_ID)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="STScI logo" width="200px"/> 